In [ ]:
import os, time, random
import numpy as np
import pandas as pd
import cv2, torch
from tqdm.auto import tqdm
import shutil as sh
import re

from IPython.display import Image, clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#%%time
#!git clone https://github.com/ultralytics/yolov5  # clone repo
#!pip install -U pycocotools
#!pip install -qr yolov5/requirements.txt  # install dependencies
#!cp yolov5/requirements.txt ./

In [ ]:
img_h, img_w, num_channels = (380, 676, 3)
df = pd.read_csv('./data/train_solution_bounding_boxes (1).csv')
df.rename(columns={'image':'image_id'}, inplace=True)
df['image_id'] = df['image_id'].apply(lambda x: x.split('.')[0])
print(df[df['image_id']=="vid_4_1000"])

# Annotation

### [ bbox_left , bbox_top , bbox_width , bbox_height , score , object_category , truncation , occlusion]

 - ### bbox_left	     
 
La coordonnée x du coin supérieur gauche de la boîte de délimitation prédite.
 

- ### bbox_top

La coordonnée y du coin supérieur gauche de la boîte englobante de l'objet prédit.

- ### bbox_width

La largeur en pixels de la boîte de délimitation de l'objet prédit.

- ### bbox_height

La hauteur en pixels de la boîte de délimitation de l'objet prédit.

- ### score

Le score du fichier DETECTION indique le degré de confiance de la boîte englobante prédite entourant une instance d'objet.
- 1 indique que le rectangle de délimitation est pris en compte dans l'évaluation, 
- 0 indique que le rectangle englobant sera ignoré.



- ### object_category

La catégorie d'objet indique le type d'objet annoté:
- 0 ignorées
- 1 piéton
- 2 personnes
- 3 bicyclette
- 4 voiture
- 5 fourgon
- 6 camion
- 7 tricycle
- 8 tricycle-auvent
- 9 bus
- 10 moteur
- 11 autres

- ### truncation

Le score dans le fichier de résultat DETECTION doit être défini sur la constante -1.
Le score indique le degré de parties d'objets apparaissant en dehors d'une image 

- 0 pas de troncature (ratio de troncature 0%)
- 1 troncature partielle (ratio de troncature 1% ~ 50%)

- ### occlusion

Le score dans le fichier GROUNDTRUTH indique la fraction d'objets occultés 

- 0 aucune occlusion (rapport d'occlusion de 0 %)
- 1 occlusion partielle (rapport d'occlusion de 1 % ~ 50 %)
- 2 forte occlusion (taux d'occlusion 50 % ~ 100 %)

In [ ]:
f = open('../data/aiskyeye_dataset/VisDrone2019-DET-train/annotations/0000008_02499_d_0000041.txt', 'r')
f.readlines()


In [ ]:
chemin_df = "../data/aiskyeye_dataset/VisDrone2019-DET-train/annotations/"
#train_images = os.path.join(train_base_dir, 'images')
train_annotations = os.path.join('../data/aiskyeye_dataset/VisDrone2019-DET-train', 'annotations')
train_annotations_list = sorted(os.listdir(train_annotations))

df_result = pd.DataFrame(columns=('image_id','bbox_left' , 'bbox_top' , 'bbox_width' , 'bbox_height' , 'score' , 'object_category' , 'truncation' , 'occlusion'))
for i in train_annotations_list:
    fichier = i
    df = open(chemin_df+fichier, "r")
    lines = df.readlines()
    df.close()
    file = fichier.split('.')[0]
    for index, line in enumerate(lines):
        lines[index] = line.strip()
    i = 0
    for line in lines:
        splits = line.split(',')
        df_result = df_result.append({'image_id': file,
                                      'bbox_left': splits[0],
                                      'bbox_top': splits[1],
                                      'bbox_width': splits[2],
                                      'bbox_height': splits[3],
                                      'score': splits[4],
                                      'object_category': splits[5],
                                      'truncation': splits[6],
                                      'occlusion': splits[7]}, ignore_index=True)
print(df_result)

In [ ]:
print(df_result)

In [ ]:
# 0 ignorées
# 1 piéton
# 2 personnes
# 3 bicyclette
# 4 voiture
# 5 fourgon
# 6 camion
# 7 tricycle
# 8 tricycle-auvent
# 9 bus
# 10 moteur
# 11 autres
def colors(x):
    return {
        0: (240, 195, 0),
        1: (0,255,0),
        2: (132, 46, 27),
        3: (131, 166, 151),
        4: (0,0,255),
        5: (223, 115, 255),
        6: (255,0,0),
        7: (255, 0, 127),
        8: (112, 41, 99),
        9: (255, 0, 127),
        10:(210, 202, 236),
        11:(0, 86, 27)
    }[x]

In [ ]:
def draw_boxes(img, box):
    
    box = [int(x) for x in box.split(',')]
    top_left_x = int(box[0])
    top_left_y = int(box[1])
    
    width = int(box[2])
    height = int(box[3])
    bottom_right_x = top_left_x + width
    bottom_right_y = top_left_y + height
    
    color = colors(int(box[5]))
    
    
    
    
    cv2.rectangle(img, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y),color, 3)